# REI505M Machine Learning - Final project
### Due: --------

**Names**: Axel Kristján Axelsson, Bjarni Haukur Bjarnason <br />
**Email**: aka30@hi.is, bhb23@hi.is

**1. Image classification**

s


In [68]:
import numpy as np
from sklearn.model_selection import train_test_split
from PIL import Image
import tensorflow as tf
import tensorflow_io as tfio


from data_loader import MyDataLoader
from neural_nets import NeuralNets
from helper_funcs import *
from my_dataset import parse_to_image



%load_ext autoreload 
%autoreload 2

norm = (128,128)

dl = MyDataLoader("combine3", norm)
nn = NeuralNets(norm)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [69]:
filenames = dl.get_all_filenames()
filenames.shape


train\y\0\buttercup2.jpg


In [82]:
BATCH_SIZE = 128
PREFETCH_SIZE = 16
TEST_SIZE = 1000
VAL_SIZE = 2000

filenames = dl.get_all_filenames()
ds = tf.data.Dataset.from_tensor_slices(filenames)

ds = ds.shuffle(buffer_size=len(filenames))

ds = ds.map(parse_to_image, num_parallel_calls=8)

ds = ds.prefetch(PREFETCH_SIZE)

ds_test  = ds.take(TEST_SIZE)
ds_val   = ds.skip(TEST_SIZE).take(VAL_SIZE)
ds_train = ds.skip(TEST_SIZE + VAL_SIZE)

ds_train = ds_train.batch(BATCH_SIZE)

In [83]:
model = nn.model_f()
model.compile(optimizer='rmsprop', loss='mse', metrics=["accuracy"])
model.fit(ds_train, epochs=10, batch_size=32, validation_data=ds_val, workers=16)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_11 (Conv2D)          (None, 128, 128, 64)      640       
                                                                 
 up_sampling2d_4 (UpSampling  (None, 256, 256, 64)     0         
 2D)                                                             
                                                                 
 conv2d_12 (Conv2D)          (None, 256, 256, 64)      36928     
                                                                 
 conv2d_13 (Conv2D)          (None, 256, 256, 128)     73856     
                                                                 
 up_sampling2d_5 (UpSampling  (None, 512, 512, 128)    0         
 2D)                                                             
                                                                 
 conv2d_14 (Conv2D)          (None, 512, 512, 128)    

In [84]:
model.fit(ds_train, epochs=10, batch_size=32, validation_data=ds_val, workers=16)

Epoch 1/10


ResourceExhaustedError:  OOM when allocating tensor with shape[128,128,256,256] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node sequential_1/conv2d_13/Relu
 (defined at C:\Users\bjarn\anaconda3\envs\tf2.6.0\lib\site-packages\keras\backend.py:4867)
]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.
 [Op:__inference_train_function_4784]

Errors may have originated from an input operation.
Input Source operations connected to node sequential_1/conv2d_13/Relu:
In[0] sequential_1/conv2d_13/BiasAdd (defined at C:\Users\bjarn\anaconda3\envs\tf2.6.0\lib\site-packages\keras\layers\convolutional.py:264)

Operation defined at: (most recent call last)
>>>   File "C:\Users\bjarn\anaconda3\envs\tf2.6.0\lib\runpy.py", line 197, in _run_module_as_main
>>>     return _run_code(code, main_globals, None,
>>

In [9]:
image = tf.io.read_file("model_e1.jpg")
image = tf.image.decode_jpeg(image)
image = tf.image.convert_image_dtype(image, tf.float16)
image = tfio.experimental.color.rgb_to_lab(image)
print(image[:,:,1].shape)

(128, 128)


In [11]:
dl.normalize_train_data(norm_size=norm)

try:
    del X
    del y

except:
    pass

X, y = dl.get_lab_data()
dl.numpy_dump(X, "X")
dl.numpy_dump(y, "y")

100%|██████████| 112/112 [05:15<00:00,  2.82s/it]


In [13]:
# X = np.squeeze(dl.numpy_load("X"))
X = dl.numpy_load("X") # 0-1
y = dl.numpy_load("y") # -1 - 1
X_lab = X
y_lab = map_to(y)

In [ ]:
dl.all_to_one()

In [3]:
minmax = np.zeros(shape=(1,1,2,2))
minmax[0,0,0,0] = np.max(y[:,:,:,0])
minmax[0,0,1,0] = np.min(y[:,:,:,0])
minmax[0,0,0,1] = np.max(y[:,:,:,1])
minmax[0,0,1,1] = np.min(y[:,:,:,1])

# print(np.max(y))
# print(np.min(y))
# print(np.max(y_lab))
# print(np.min(y_lab))
# test = map_from(y_lab, y)
# print(np.max(test))
# print(np.min(test))

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X_lab, y_lab, test_size = 0.3, random_state = 3)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size = 0.33, random_state = 3)

In [ ]:
# from random import randint
from tensorflow.keras.optimizers import *
from tensorflow.keras.losses import *

# ?GANS svona
gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.8)
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))

log_dir = "logs/fit/" + "model_e"
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

#model = load_model("model_e")
model = nn.autoencoder()
model.compile(optimizer=RMSprop(), loss=MeanSquaredError())


percent = 0.6
size = X_train.shape[0]
for i in range(10):
    rand_array = np.random.choice(size, int(percent*size))
    X_bla = X_train[rand_array]
    y_bla = y_train[rand_array]
    model.fit(X_bla, y_bla, batch_size = 64, epochs = 5, validation_data=(X_val, y_val), callbacks=[tensorboard_callback])#, callbacks=[callback])
#save_model(model, "model_e", brave=True)



In [ ]:
%tensorboard --logdir logs/fit/

In [ ]:
percent = 1
size = X_train.shape[0]
rand_array = np.random.choice(size, int(percent*size))
X_bla = X_train[rand_array]

train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))

In [5]:
from random import randint

log_dir = "logs/fit/" + "model_e_2"
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# gpus = tf.config.list_physical_devices('GPU')
# if gpus:
#         for gpu in gpus:
#             tf.config.experimental.set_memory_growth(gpu,True)


model = nn.model_f()
model.compile(optimizer='rmsprop', loss='mse', metrics=["accuracy"])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 128, 128, 64)      640       
_________________________________________________________________
up_sampling2d (UpSampling2D) (None, 256, 256, 64)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 256, 256, 64)      36928     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 256, 256, 128)     73856     
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 512, 512, 128)     0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 512, 512, 128)     147584    
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 512, 512, 256)     2

In [9]:
from math import floor
from sys import getsizeof
batch_size = 512
train_size = X_train.shape[0]

num_batches = floor(train_size/batch_size)

dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))

data_batches = dataset.batch(batch_size=batch_size, drop_remainder=True)

print(getsizeof(data_batches))

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

In [7]:
num_epochs = 10
for i in range(num_epochs):
    data_batches = dataset.batch(batch_size=batch_size, drop_remainder=True)

    model.fit(data_batches, epochs=1, batch_size=32, validation_data=(X_val, y_val),
                  callbacks=[tensorboard_callback], workers=16)
    

    img_number = randint(0, train_size-1)
    test_image = np.expand_dims(X_test[img_number], 0)
    pred = model.predict(test_image)
    save_images(test_image, map_from(pred, minmax), name="model_e", enumerate=i)
    save_model(model, "model_f")

ResourceExhaustedError:  OOM when allocating tensor with shape[512,128,128,64] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node sequential/conv2d/Relu-0-1-TransposeNCHWToNHWC-LayoutOptimizer}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.
 [Op:__inference_train_function_1671]

Function call stack:
train_function


In [ ]:

    # model.fit(x=X_bla, y=y_bla, epochs=10, batch_size=32, validation_data=(X_val, y_val),
    #           callbacks=[tensorboard_callback],
    #           use_multiprocessing=True, workers=64)

In [ ]:
model = load_model("new_model")
minmax = np.zeros(shape=(1,1,2,2))
minmax[0,0,0,0] = np.max(y[:,:,:,0])
minmax[0,0,1,0] = np.min(y[:,:,:,0])
minmax[0,0,0,1] = np.max(y[:,:,:,1])
minmax[0,0,1,1] = np.min(y[:,:,:,1])

In [ ]:



rand_array = np.random.choice(X_test.shape[0], 3)
pred = model.predict(X_test[rand_array])
none_arr = np.zeros(shape=(3,128,128,2))
show_images(X_test[rand_array], none_arr)
show_images(X_test[rand_array], map_from(pred, minmax), enhance=True)

In [ ]:
save_model(model, "model_e_3", brave=True)


In [ ]:
# modell = load_model("model_e")
# none_arr = np.zeros(shape=(3, 128, 128, 2))
# modell = load_model("model_e_2")
rand_array = np.random.choice(X_test.shape[0], 3)
pred = model.predict(X_test[rand_array])
show_images(X_test[rand_array], map_from(y_test[rand_array], minmax))
show_images(X_test[rand_array], map_from(pred, minmax), enhance=True)
# save_images(X_test[rand_array], map_from(pred, y), name="pred", enhance=True)
# save_images(X_test[rand_array], map_from(y_test[rand_array], y), name="true")

In [ ]:
print(pred.shape)
print(np.max(pred))
print(np.min(pred))
new = map_from(pred, y_train)
print(np.max(new))
print(np.min(new))
# show_images(X_test[rand_array], map_from(pred, y_train))

In [ ]:
print(X_lab.shape)
print(y_lab.shape)
print(np.max(X_lab))
print(np.max(y_lab))
print(np.min(X_lab))
print(np.min(y_lab))


In [ ]:
datagen = ImageDataGenerator(rotation_range=30, fill_mode='nearest',
                             width_shift_range=0.2, height_shift_range=0.2,
                             horizontal_flip=True, vertical_flip=True,
                             brightness_range=[0.4,1.5],
                             
)

In [ ]:
from helper_funcs import *
arr = y_train[:10]
print(type(arr[0,0,0,0]))
print(np.max(arr))
print(np.min(arr))

test = map_to(arr)
print(type(test[0,0,0,0]))
print(np.max(test))
print(np.min(test))

In [ ]:
baka = map_from(test, arr)
print(type(baka[0,0,0,0]))
print(np.max(baka))
print(np.min(baka))

In [ ]:

images = np.zeros(shape=(10, 128, 128, 3))
images[:,:,:,0] = np.squeeze(X_train[:10], -1)
images[:,:,:,1:] = y_train[:10]
datagen = ImageDataGenerator(rotation_range=30, fill_mode='nearest',
                             width_shift_range=0.2, height_shift_range=0.2,
                             horizontal_flip=True, vertical_flip=True
)
image_it = datagen.flow(images, batch_size=10, shuffle=True)

X_next = image_it.next()[:,:,:,0]
y_next = image_it.next()[:,:,:,1:]
X_bla = np.expand_dims(np.array(X_next), -1)
y_bla = np.array(y_next)
print(X_bla.shape)
print(y_bla.shape)
print(np.max(y_bla))
print(np.min(y_bla))
y_bla = map_from(y_bla, y)
print(np.max(y_bla))
print(np.min(y_bla))
show_images(X_bla, y_bla)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import *
from keras import backend as K

log_dir = "logs/fit/" + "model_e_2"
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)





img_number = 42
test_image = np.expand_dims(X_test[img_number], 0)

data_gen_args = dict(rotation_range=30, fill_mode='nearest',
                     width_shift_range=0.2, height_shift_range=0.2,
                     horizontal_flip=True, vertical_flip=True,
                     shear_range=0.3
)
image_datagen = ImageDataGenerator(**data_gen_args)
mask_datagen = ImageDataGenerator(**data_gen_args)

seed = 1337
image_datagen.fit(X_train, augment=True, seed=seed)
mask_datagen.fit(y_train, augment=True, seed=seed)

image_generator = image_datagen.flow(X_train, seed = seed, batch_size=64)
mask_generator = image_datagen.flow(y_train, seed = seed, batch_size=64)

train_generator = zip(image_generator, mask_generator)

In [ ]:
# input_shape = (128, 128, 1)
# batch_size = 32
# kernel_size = 3
# latent_dim = 256
# layer_filters = [64, 128, 256]
# layer_filters = layer_filters[::-1]

# inputs = Input(shape = input_shape)
# x = inputs
# for filters in layer_filters:
#     x = Conv2D(filters = filters,
#         kernel_size = kernel_size,
#         strides = 2,
#         activation ='relu',
#         padding ='same')(x)
  
# shape = K.int_shape(x)
# x = Flatten()(x)
# latent = Dense(latent_dim, name ='latent_vector')(x)
# encoder = Model(inputs, latent, name ='encoder')

# latent_inputs = Input(shape =(latent_dim, ), name ='decoder_input')
# x = Dense(shape[1]*shape[2]*shape[3])(latent_inputs)
# x = Reshape((shape[1], shape[2], shape[3]))(x)
# # stack of Conv2DTranspose(256)-Conv2DTranspose(128)-
# # Conv2DTranspose(64)
# for filters in layer_filters[::-1]:
#     x = Conv2DTranspose(filters = filters,
#                         kernel_size = kernel_size,
#                         strides = 2,
#                         activation ='relu',
#                         padding ='same')(x)
#     outputs = Conv2DTranspose(filters = 2,
#                             kernel_size = kernel_size,
#                             activation ='sigmoid',
#                             padding ='same',
#                             name ='decoder_output')(x)
#     decoder = Model(latent_inputs, outputs, name ='decoder')

#     autoencoder = Model(inputs, decoder(encoder(inputs)),
#                     name ='autoencoder')

In [ ]:
# data_gen_args = dict(rotation_range=30, fill_mode='nearest',
#                      width_shift_range=0.2, height_shift_range=0.2,
#                      horizontal_flip=True, vertical_flip=True,
#                      shear_range=0.3
# )
# image_datagen = ImageDataGenerator(**data_gen_args)
# mask_datagen = ImageDataGenerator(**data_gen_args)

# seed = 1337
# image_datagen.fit(X_train, augment=True, seed=seed)
# mask_datagen.fit(y_train, augment=True, seed=seed)

# image_generator = image_datagen.flow(X_train, seed = seed, batch_size=64)
# mask_generator = image_datagen.flow(y_train, seed = seed, batch_size=64)

# train_generator = zip(image_generator, mask_generator)


# # model = nn.autoencoder()
# # model.compile(optimizer=RMSprop(), loss=MeanSquaredError(), metrics=["accuracy"])
# # model = nn.model_s()
# # model.compile(optimizer='rmsprop', loss='mse', metrics=["accuracy"])